<h1 style="font-size: 3em;">Application de détection de faux billets</h1>

# Importation des librairies

In [1]:
import pandas as pd
import os
import joblib

### Fonction de chargement des données

In [7]:
def load_data(file_path, delimiter=',', **kwargs):
   
    # Obtenir l'extension du fichier
    _, file_extension = os.path.splitext(file_path)
    
    # Charger le fichier en fonction de son extension
    if file_extension == '.csv':
        data = pd.read_csv(file_path, delimiter=delimiter, **kwargs)
    elif file_extension in ['.xls', '.xlsx']:
        data = pd.read_excel(file_path, **kwargs)
    elif file_extension == '.json':
        data = pd.read_json(file_path, **kwargs)
    elif file_extension == '.parquet':
        data = pd.read_parquet(file_path, **kwargs)
    elif file_extension == '.txt':
        # Si le fichier texte utilise le délimiteur fourni
        data = pd.read_csv(file_path, delimiter=delimiter, **kwargs)
    else:
        raise ValueError(f"Extension de fichier '{file_extension}' non supportée.")
    # Spécifiez les colonnes nécessaires pour le modèle
    columns_needed = ['id','diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
    
    # Garder seulement les colonnes nécessaires si elles sont présentes dans le DataFrame
    data = data[[col for col in columns_needed if col in data.columns]]
    return data



### Fonction de prédiction

In [8]:
def predict_new_data(model, new_data):
    
    # Séparer les IDs des données pour la prédiction
    ids = new_data['id']
    data_for_prediction = new_data.drop(columns=['id'])

    # Effectuer les prédictions
    predictions = model.predict(data_for_prediction)

    # Mapper les prédictions pour obtenir "Vrai" ou "Faux"
    new_data['Prediction'] = predictions
    new_data['Prediction'] = new_data['Prediction'].map({1: 'Vrai', 0: 'Faux'})

    # Retourner un DataFrame avec uniquement les colonnes 'id' et 'Prediction'
    return new_data[['id', 'Prediction']]


### Fonction chargement et prédiction

In [9]:
def load_and_predict(file_path, model_path, delimiter=',', **kwargs):
   
    
    # Charger les données
    new_data = load_data(file_path, delimiter=delimiter, **kwargs)

    # Charger le modèle sauvegardé
    model = joblib.load(model_path)
    
    # Prédire les nouvelles données
    predictions = predict_new_data(model, new_data)

    return predictions

# Application de la fonction

In [16]:
# Chemin vers le fichier contenant les données à prédire
file_path = 'billets_production.csv'  
# Chemin vers le modèle sauvegardé
model_path = 'final_logistic_model.pkl'  


In [17]:
load_and_predict(file_path, model_path, delimiter=',')

,id,Prediction
0,A_1,Faux
1,A_2,Faux
2,A_3,Faux
3,A_4,Vrai
4,A_5,Vrai
